In [347]:
from gurobipy import *
import numpy as np
import math as mt

In [348]:
# Données
service_count=2
project_count=4
n = 2
p=4
budget_limit = 50
constraint_count=n*n +1
variable_count= n*n + p
couts_projet = np.array([40, 50, 60, 50])

In [349]:
# Matrice d'utilte
matrice_utilite = np.array([[19, 6, 17, 2], [2, 11, 4, 18]])
print(matrice_utilite)

[[19  6 17  2]
 [ 2 11  4 18]]


In [350]:
# vecteur w
w = [2, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

[1, 1]


In [351]:
# matrice utilite
u = np.array([
        [19, 6, 17, 2],
        [2, 11, 4, 18]
])
print(u)

[[19  6 17  2]
 [ 2 11  4 18]]


In [352]:
# Generation de la matrice des contraintes 

In [353]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)


[[1 0]
 [1 0]
 [0 1]
 [0 1]]


In [354]:
## les variables bik
coef_bik = np.identity(n**2)
coef_bik *= -1
print(coef_bik)

[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]


In [355]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
print(coef_xi)

[[-19, -6, -17, -2], [-2, -11, -4, -18], [-19, -6, -17, -2], [-2, -11, -4, -18]]


In [356]:
# contrainte sur le budget
budget = [0 for i in range(n*(n+1))]
budget += [couts_projet[i] for i in range(p)]
budget = np.array(budget)
print(budget)

[ 0  0  0  0  0  0 40 50 60 50]


In [357]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)

# ajout de la contrainte sur les budgets
constrainte = np.vstack([result_concat, budget])
print(constrainte)


[[  1.   0.  -1.  -0.  -0.  -0. -19.  -6. -17.  -2.]
 [  1.   0.  -0.  -1.  -0.  -0.  -2. -11.  -4. -18.]
 [  0.   1.  -0.  -0.  -1.  -0. -19.  -6. -17.  -2.]
 [  0.   1.  -0.  -0.  -0.  -1.  -2. -11.  -4. -18.]
 [  0.   0.   0.   0.   0.   0.  40.  50.  60.  50.]]


In [358]:
## second memebre
second_membre = np.array(
        [0 for i in range(n**2)].append (budget_limit)
)
print (second_membre)

[0 0 0 0 0]


In [359]:
# Coefficient de la fonction objectif
## vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])

## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
c += [- w_prime[i%n]  for i in range(0, n*n)] # coef des b_ik
c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)

In [360]:
# Creer les models de la fonction objectif
m = Model("exercice1.4")

In [361]:
# declaration des variables de decisions
x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((i+1,(j+1)))))
for j in range(p):
    x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))

In [362]:
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var r1>, <gurobi.Var r2>, <gurobi.Var b11>, <gurobi.Var b12>, <gurobi.Var b21>, <gurobi.Var b22>, <gurobi.Var x1>, <gurobi.Var x2>, <gurobi.Var x3>, <gurobi.Var x4>]


In [363]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(variable_count):
    obj += c[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [364]:
# Definition des contraintes
# Definition des contraintes
lignes = range(constraint_count)
colonnes = range(variable_count)
print (lignes, colonnes)
for i in lignes[:-1]:
    m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)
    
m.addConstr(quicksum(constrainte[-1][j]*x[j] for j in colonnes) <= second_membre[-1], "Contrainte%d" % i)

range(0, 5) range(0, 8)


<gurobi.Constr *Awaiting Model Update*>